In [34]:
import glob
import os
from PIL import Image,ImageDraw

## 自己寫的函式庫(匯入使用)

In [15]:
import CompareImage
import Harr
file='Screen/scan228.jpg'
file2='Screen/scan226.jpg'
print(CompareImage.CompareAlgo(file,file2))

False


## BubbleSort Test

In [16]:
def bubbleSort(arr): 
    n = len(arr) 
    # Traverse through all array elements 
    for i in range(n-1): 
    # range(n) also work but outer loop will repeat one time more than needed. 
    # Last i elements are already in place 
        for j in range(0, n-i-1): 
            
            # traverse the array from 0 to n-i-1 
            # Swap if the element found is greater 
            # than the next element 
            if arr[j] > arr[j+1] : 
                arr[j], arr[j+1] = arr[j+1], arr[j] 


In [17]:
# Driver code to test above 
arr = [64, 34, 25, 12, 22, 11, 90] 
  
bubbleSort(arr) 
  
print ("Sorted array is:") 
for i in range(len(arr)): 
    print ("%d" %arr[i])

Sorted array is:
11
12
22
25
34
64
90


## 兩兩截圖比對

In [33]:
#比對等待(等待秒,截圖數量)
def sleep_compare(sec):
    #讀進前面截圖的資料
    filepath='Screen\\Test\\'
    myfiles=glob.glob(filepath+'/*.jpg')
    #宣告字典
    ImageArray=[]
    for i,f in enumerate(myfiles):
        #放到Arraay排隊
        ImageArray.append(f)
        print("檔案",ImageArray[i])
    #不能用len(Imagelist) 因int是不可以疊帶
    for i in ImageArray:
        #算數量
        count=0
        count+=1
    
    print("檔案",ImageArray)

    #判斷裡面沒大於2張無法比較
    if len(ImageArray)<=1:
        print("End")
    #如果超過兩張就兩兩比較
    if len(ImageArray)>=2:
        print("Start")
        #兩兩拉出 1跟2比，2跟3比，3跟4比
        N=len(ImageArray)-1
        #從i=0，j=0開始
        for  i in range(N):
            #for j in range(0,N-i-1):
                #判斷裡面沒大於2張無法比較
                if len(ImageArray)<1:
                    print("End 結束")
                else:
                    print("比對 開檔",i)
                    #取檔案順序
                    File1=i
                    File2=i+1

                    #從隊伍撈出來開檔案
                    img1=Image.open(ImageArray[File1])
                    print('//1//',ImageArray[File1])
                    #取檔名
                    File1Name=str(ImageArray[File1]).split("\\")[-1]
                    #停頓
                    #await asyncio.sleep(sec)
                    #img1.show()
                    #刪掉list第一個值(被開的檔名)
                    #ImageArray.remove(str(ImageArray[File1]))
                    img2=Image.open(ImageArray[File2])
                    print('//2//',ImageArray[File2])
                    #取檔名
                    File2Name=str(ImageArray[File1]).split("\\")[-1]
                    #刪掉list第一個值(被開的檔名)
                    #ImageArray.remove(str(ImageArray[File2]))
                    #img2.show()
                    print("取檔案OK，開始比對圖片")
                    rate=CompareImage.CompareAlgo(img1,img2)
                    #比對相似度
                    if(CompareImage.CompareAlgo(img1,img2)==True):
                        
                        RemoveFile=ImageArray[File1]
                        try:
                            os.remove(RemoveFile)
                        except OSError as e:
                            print(e)
                        else:
                            print("刪除",File1Name)
                    else:
                        print("不相同")
                        
                        
    print("i結束",i)
                        
                    
    print(ImageArray)
        
        

SyntaxError: invalid syntax (<ipython-input-33-ab74ba2f87ac>, line 44)

In [32]:
sleep_compare(1)

檔案 Screen\Test\scan001.jpg
檔案 Screen\Test\scan002.jpg
檔案 Screen\Test\scan003.jpg
檔案 Screen\Test\scan004.jpg
檔案 Screen\Test\scan005.jpg
檔案 Screen\Test\scan006.jpg
檔案 Screen\Test\scan007.jpg
檔案 ['Screen\\Test\\scan001.jpg', 'Screen\\Test\\scan002.jpg', 'Screen\\Test\\scan003.jpg', 'Screen\\Test\\scan004.jpg', 'Screen\\Test\\scan005.jpg', 'Screen\\Test\\scan006.jpg', 'Screen\\Test\\scan007.jpg']
Start
比對 開檔 0
//1// Screen\Test\scan001.jpg
檔名 scan001.jpg
//2// Screen\Test\scan002.jpg
OK
比對 開檔 1
//1// Screen\Test\scan002.jpg
檔名 scan002.jpg
//2// Screen\Test\scan003.jpg
OK
比對 開檔 2
//1// Screen\Test\scan003.jpg
檔名 scan003.jpg
//2// Screen\Test\scan004.jpg
OK
比對 開檔 3
//1// Screen\Test\scan004.jpg
檔名 scan004.jpg
//2// Screen\Test\scan005.jpg
OK
比對 開檔 4
//1// Screen\Test\scan005.jpg
檔名 scan005.jpg
//2// Screen\Test\scan006.jpg
OK
比對 開檔 5
//1// Screen\Test\scan006.jpg
檔名 scan006.jpg
//2// Screen\Test\scan007.jpg
OK
i結束 5
['Screen\\Test\\scan001.jpg', 'Screen\\Test\\scan002.jpg', 'Screen\\Test\\sc